In [ ]:
%%capture
%reset -sf
%load_ext autoreload
%autoreload 2
!pip install --user kaggle-environments > /dev/null

In [ ]:
!rm -f *.py *.pickle *.pth

from IPython.core.magic import register_cell_magic

@register_cell_magic
def writefile_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = 'w'
    if len(argz) == 2 and argz[0] == '-a':
        mode = 'a'
    with open(file, mode) as f:
        f.write(cell)
    get_ipython().run_cell(cell)

In [ ]:
!cp ../input/kore-2022-match-analysis/kore_analysis.py .
!cp ../input/kore-2022-imitation-training/feature_generator.py .  # actually from feature-generator notebook
!cp ../input/kore-2022-imitation-training/model.pth .
!cp ../input/kore-2022-imitation-training/imitation_training_helper.py .

In [ ]:
%%writefile_and_run -a main.py
import os, re, json, enum, glob, shutil, collections, requests, pickle, math, random

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.animation
import matplotlib.patheffects
import matplotlib.pyplot as plt
import IPython.display

import torch
from scipy.special import softmax

import kaggle_environments
from kaggle_environments.envs.kore_fleets.helpers import Board

from kore_analysis import KoreMatch, load_from_episode_id, plot_3d_matrix, plot_dataframe_of_3d_points, existence_to_production_capacity
from feature_generator import parse_env
from imitation_training_helper import append_source_specific_features

torch_device = "cuda" if torch.cuda.is_available() else "cpu"

def debug(item):
    with open('z.log', "a") as f:
        f.write(str(item))
        f.write("\n")

In [ ]:
final_env = load_from_episode_id(36620119)
acting_player_id = 1

In [ ]:
def load_turn_from_replay_json(env_final, turn_idx, player_id):    
    match_state = env_final.steps[turn_idx]
    for player_id in [0,1]:
        match_state[player_id]["observation"]["remainingOverageTime"] \
            = max(0, match_state[player_id]["observation"]["remainingOverageTime"])
    env = kaggle_environments.make("kore_fleets", steps=[match_state],
                                   configuration=env_final.configuration)
    return env

In [ ]:
env = load_turn_from_replay_json(final_env, 11, acting_player_id)
env.steps[0][acting_player_id]['action']

In [ ]:
for _ in range(12):
    if not env.done:
        env.step([{}, {}])
    else:
        env.steps.append(env.steps[-1])

input_matrix = parse_env(env, acting_player_id=1)
input_matrix.shape

In [ ]:
%%writefile_and_run -a main.py

path = '/kaggle_simulations/agent' if os.path.exists('/kaggle_simulations') else '.'
model = torch.jit.load(f'{path}/model.pth')
model.eval()
pass

In [ ]:
%%writefile_and_run -a main.py

def generate_flight_plan_with_action_tuple(action_class, diff_x, diff_y, polarity_if_build=True):
    flight_plan = ""
    constructing = action_class == 0
    if action_class == 0:  # construct
        if polarity_if_build:
            action_class = 1  # or 2, does not matter
        else:
            action_class = 2

    x_move_up = "E" + str(abs(diff_x)-1)
    y_move_up = "N" + str(abs(diff_y)-1)
    
    x_move_down = "W" + str(abs(diff_x)-1)
    y_move_down = "S" + str(abs(diff_y)-1)
    
    move_arr = [""] * 4
    
    if diff_x > 0:
        move_arr[0] = x_move_up
        move_arr[3] = x_move_down  
    
    elif diff_x < 0:
        move_arr[3] = x_move_up
        move_arr[0] = x_move_down
        
    if diff_y > 0:
        move_arr[1] = y_move_up
        move_arr[2] = y_move_down  
    
    elif diff_y < 0:
        move_arr[2] = y_move_up
        move_arr[1] = y_move_down
        
    # action == 1 -> polarity == True -> start with N/S
    if action_class == 1:
        move_arr[0], move_arr[1] = move_arr[1], move_arr[0]
        move_arr[2], move_arr[3] = move_arr[3], move_arr[2]

    if constructing:
        move_arr = move_arr[:2] + ["C"] + move_arr[2:]
    
    move_arr = [x for x in move_arr if x != ""]
    if not move_arr:
        return ""
    move_arr[-1] = move_arr[-1][0]  # last digit is not required

    flight_plan = "".join(move_arr).replace("0", "")
    return flight_plan

In [ ]:
%%writefile_and_run -a main.py

turn_idx = 0

def get_best_flight_plan(policy, input_matrix, limit=9, force_build=False):
    global turn_idx
    max_flight_plan = ""
    flight_plans_and_details = []
    
    for clf_idx, prob in enumerate(policy):
        action_class, clf_idx = divmod(clf_idx, 21*21)
        diff_x_shifted, diff_y_shifted = divmod(clf_idx, 21)
        
        if action_class == 0:  # assume polarity is True
            kore_density = input_matrix[0, diff_x_shifted, diff_y_shifted]
        if action_class == 1:
            kore_density = input_matrix[0, diff_x_shifted, diff_y_shifted]
        if action_class == 2:
            kore_density = input_matrix[1, diff_x_shifted, diff_y_shifted]

        diff_x = diff_x_shifted - 10
        diff_y = diff_y_shifted - 10
        del diff_x_shifted, diff_y_shifted
        
#         if (abs(diff_x) == 0 or abs(diff_y) == 0) and turn_idx <= 111:
#             continue  # do not attack opponent starter base from your starter base
#         if force_build and action_class != 0:  # if force_build, must build
#             continue
#         if not force_build and action_class == 0:  # if no force_build, no build
#             continue
        if diff_x == 0 and diff_y == 0:
            continue  # skip impossible actions
        
        flight_plan = generate_flight_plan_with_action_tuple(action_class, diff_x, diff_y)
#         if kore_density < 0.1 * turn_idx / 400:  # avoid pointless inter-shipyard transfers
#             continue
        if len(flight_plan) > limit:
            continue
        if "C" in flight_plan and limit <= 8:
            continue
        
        flight_plans_and_details.append({
            'prob': prob,
            'flight_plan': flight_plan,
            'kore_density': kore_density,
            'diff_x': diff_x,
            'diff_y': diff_y,
        })
        
    if not flight_plans_and_details:
        debug("no available plan")
        return ""
    
    flight_plans_and_details.sort(key=lambda x:x['prob'], reverse=True)
    flight_plans_and_details = flight_plans_and_details[:8]  # consider only the 8 most probable plans
    flight_plan_and_detail = random.choices(
        population=flight_plans_and_details,
        weights=[x['kore_density']*x['prob'] for x in flight_plans_and_details],
        k=1
    )[0]
    debug(flight_plans_and_details[0])  # maxprob
    debug(flight_plan_and_detail)  # selected
    
    return flight_plan_and_detail['flight_plan']

In [ ]:
%%writefile_and_run -a main.py

def get_flight_plan(shipyard_x, shipyard_y, input_matrix, limit=10, force_build=False):
    global viz_input_matrix, viz_output_matrix
    input_matrix = np.roll(input_matrix, (0, -shipyard_x+10, -shipyard_y+10), axis=(0,1,2))  
    input_matrix = append_source_specific_features(input_matrix)

    viz_input_matrix = input_matrix
    
    with torch.no_grad():
        p = model(torch.from_numpy(input_matrix.copy()).unsqueeze(0))
    p = softmax(p)
    viz_output_matrix = p.numpy().reshape(3,21,21)
    policy = p.squeeze(0).numpy()
    
    # assert input_matrix[-12,10,10] != 0
    assert input_matrix[0,10,10] == 0
    assert input_matrix[1,10,10] == 0

    return get_best_flight_plan(policy, input_matrix, limit=limit, force_build=force_build)

In [ ]:
loc_idx = list(env.steps[0][0]["observation"]["players"][1][1].values())[0][0]
sx,sy = kaggle_environments.helpers.Point.from_index(int(loc_idx), 21)
get_flight_plan(sx,sy,input_matrix,limit=9,force_build=True)

In [ ]:
assert viz_input_matrix[-2,10,10] != 0
viz_input_matrix.shape

In [ ]:
plot_3d_matrix(viz_input_matrix)

In [ ]:
viz_output_matrix.shape

In [ ]:
def parse_output_matrix_into_dataframe(matrix):
    dict_list = []
    xrr, yrr, zrr, crr = [], [], [], []
    nx, ny, nz = matrix.shape
    for x in range(nx):
        for y in range(ny):
            for z in range(nz):
                val = matrix[x,y,z]
                if val > 10**-9:
                    dict_item = {
                        'x': x,
                        'y': y,
                        'z': z,
                        'value': val,
                        'plan': generate_flight_plan_with_action_tuple(x,y-10,z-10)
                    }
                    dict_list.append(dict_item)
    matrix_dataframe = pd.DataFrame.from_records(dict_list)
    return matrix_dataframe

import plotly
def plot_dataframe_of_3d_points(input_dataframe):
    # to add an option in kore_analysis.py
    plotly.offline.init_notebook_mode()
    
    range_color_upper = max(0, min(1, max(input_dataframe['value'])))
    
    fig = plotly.express.scatter_3d(input_dataframe, x='x', y='y', z='z',
              color='value', range_color=[0,range_color_upper],
              opacity=0.7, hover_data=input_dataframe.columns)

    fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
    fig.show()

In [ ]:
plot_dataframe_of_3d_points(parse_output_matrix_into_dataframe(viz_output_matrix))

In [ ]:
def plot_dataframe_of_3d_points(input_dataframe, x_colname='x', y_colname='y', z_colname='z', 
                                range_color_upper=None, range_y=[0,20], range_z=[0,20], 
                                scene_camera_eye=dict(x=0, y=-1, z=-1),
                                color_colname='value', symbol_colname=None, size_colname=None):
    plotly.offline.init_notebook_mode()
    
    if range_color_upper == None:
        range_color_upper = max(0, min(1, max(input_dataframe[color_colname])))
    
    fig = plotly.express.scatter_3d(
        input_dataframe, x=x_colname, y=y_colname, z=z_colname,
        range_x=[0,max(input_dataframe[x_colname])], range_y=range_y, range_z=range_z,
        symbol=symbol_colname, size=size_colname,
        color=color_colname, range_color=[0,range_color_upper], color_continuous_scale='bluered',
        opacity=0.7, hover_data=input_dataframe.columns)
    
    fig.update_layout(margin=dict(l=0, r=0, b=0, t=0), 
                      scene_camera=dict(eye=scene_camera_eye), dragmode='pan')
    fig.show()

In [ ]:
env.steps[0][acting_player_id]['action']

In [ ]:
%%writefile_and_run -a main.py
def get_maximum_flight_plan_length(fleet_size, plan_length_limit=9):
    if fleet_size == 0: return 0
    plan_length = math.floor(2 * math.log(fleet_size)) + 1
    plan_length = min(plan_length_limit, plan_length)
    return plan_length


def get_minimum_fleetsize_for_flightplan(flight_plan):
    if len(flight_plan) >= 9: return 55
    if "C" in flight_plan: return 50
    if len(flight_plan) >= 7: return 21  # maybe add one to break ties
    if len(flight_plan) >= 6: return 13
    if len(flight_plan) >= 5: return 8
    if len(flight_plan) >= 4: return 5
    if len(flight_plan) >= 3: return 3
    if len(flight_plan) >= 2: return 2
    if len(flight_plan) >= 1: return 1
    return 0

In [ ]:
%%writefile_and_run -a main.py

turn_idx = 0
def agent(observation, configuration):
    global turn_idx, viz_output_matrix
    try:
        env = kaggle_environments.make("kore_fleets", configuration=configuration)
        env.reset(num_agents=2)
        env.steps[-1][0]["observation"] = observation
        turn_idx = observation['step']

        for _ in range(21):
            if not env.done:
                env.step([{}, {}])
            else:
                env.steps.append(env.steps[-1])
                
        input_matrix = parse_env(env, observation.player)
        kore_match = KoreMatch(env.steps)
    
        # count how many ships, to calculate available fleet
        loc_idx_to_minimum_ship_count_home = collections.defaultdict(lambda: 10_000)
        loc_idx_to_minimum_ship_count_away = collections.defaultdict(lambda: 10_000)
        for home_shipyards, away_shipyards in zip(kore_match.home_shipyards[1:], kore_match.away_shipyards[1:]):
            for loc_idx, ship_count, _ in home_shipyards.values():
                loc_idx_to_minimum_ship_count_home[loc_idx] = min(loc_idx_to_minimum_ship_count_home[loc_idx], ship_count)
                loc_idx_to_minimum_ship_count_away[loc_idx] = min(loc_idx_to_minimum_ship_count_home[loc_idx], -ship_count)
            for loc_idx, ship_count, _ in away_shipyards.values():
                loc_idx_to_minimum_ship_count_away[loc_idx] = min(loc_idx_to_minimum_ship_count_away[loc_idx], ship_count)
                loc_idx_to_minimum_ship_count_home[loc_idx] = min(loc_idx_to_minimum_ship_count_away[loc_idx], -ship_count)
    
        if observation.player == 0:
            shipyards = kore_match.home_shipyards[0]
            kore_stored = kore_match.home_kore_stored[0]
            has_fleets_deployed = bool(kore_match.home_fleets[0])
            opponent_kore_stored = kore_match.away_kore_stored[0]
            loc_idx_to_minimum_ship_count = loc_idx_to_minimum_ship_count_home
        else:
            shipyards = kore_match.away_shipyards[0]
            kore_stored = kore_match.away_kore_stored[0]
            has_fleets_deployed = bool(kore_match.away_fleets[0])
            opponent_kore_stored = kore_match.home_kore_stored[0]
            loc_idx_to_minimum_ship_count = loc_idx_to_minimum_ship_count_away

        actions = {}
        forced_build = False
        for shipyard_idx, (loc_idx,ship_count,turn_existence) in shipyards.items():
            production_capacity = existence_to_production_capacity(turn_existence)
            shipyard_x,shipyard_y = kaggle_environments.helpers.Point.from_index(int(loc_idx), 21)
            
            available_fleet = max(0, loc_idx_to_minimum_ship_count[loc_idx])
            maximum_flight_plan_length = get_maximum_flight_plan_length(min(ship_count, available_fleet))

            force_build = (available_fleet >= 50 and 
                           kore_stored >= 200 and 
                           available_fleet*10 + kore_stored >= 1500 and 
                           not forced_build)
            forced_build = forced_build or force_build
            
            flight_plan = get_flight_plan(shipyard_x,shipyard_y,input_matrix,maximum_flight_plan_length,force_build=force_build)
            minimum_ships_to_launch = get_minimum_fleetsize_for_flightplan(flight_plan)
            
            debug(("inferred flight_plan", flight_plan))

            if "C" in flight_plan:
                minimum_ships_to_launch = min(max(128, available_fleet//2),  # avoid coalescing 
                                              max(minimum_ships_to_launch, available_fleet))
            if kore_stored // 10 + available_fleet >= 21:  # encourage build to 21 if possible
                minimum_ships_to_launch =  max(21, minimum_ships_to_launch)
            if minimum_ships_to_launch < 21:
                minimum_ships_to_launch = available_fleet

            if (
                (len(flight_plan) == 0) or 
                (minimum_ships_to_launch > ship_count) or 
                (minimum_ships_to_launch > available_fleet)
               ):
                ships_to_produce = min(int(kore_stored) // 10, production_capacity)
                if ships_to_produce > 0:
                    actions[shipyard_idx] = f"SPAWN_{ships_to_produce}"
                    kore_stored -= ships_to_produce*10
                continue
           
            remaining_fleet = available_fleet - minimum_ships_to_launch
            if flight_plan:
                if remaining_fleet < production_capacity:
                    minimum_ships_to_launch += remaining_fleet
                    remaining_fleet = available_fleet - minimum_ships_to_launch
                elif remaining_fleet < 21 and remaining_fleet + min(kore_stored // 10, production_capacity) > 21:
                    ships_carried_forward = remaining_fleet + production_capacity - 21
                    minimum_ships_to_launch += ships_carried_forward
                    remaining_fleet = available_fleet - minimum_ships_to_launch
                    
                minimum_ships_to_launch and kore_stored >= ship_count
                actions[shipyard_idx] = f"LAUNCH_{minimum_ships_to_launch}_{flight_plan}"

    except Exception as e:
        debug("error")
        debug(e)
        debug(f"{type(e).__name__} at line {e.__traceback__.tb_lineno} of {__file__}: {e}")
        return {}
    
    debug((turn_idx, "executed", actions))
    return actions

In [ ]:
!python main.py  # syntax check

In [ ]:
!rm z.log
!touch z.log

In [ ]:
from kaggle_environments import make

env = make("kore_fleets", debug=False)
steps = env.run(['balanced', 'main.py'])

In [ ]:
!head -10 z.log

In [ ]:
!grep -C 10 "(154," z.log

In [ ]:
# https://www.kaggle.com/code/jmerle/koreye-2022-integration/notebook

import json
from IPython import get_ipython
from IPython.display import display, HTML

def render_env(env, open_koreye=False, show_koreye_button=True, show_official_viewer=True, *args, **kwargs):
    """Renders an env and includes a button to open the episode in Koreye 2022.

    :param env: the env to render
    :param open_koreye: whether the episode should be opened in Koreye 2022 immediately (only while editing the notebook)
    :param show_koreye_button: whether a button to open the episode in Koreye 2022 should be shown
    :param show_official_viewer: whether the official episode viewer should be shown
    :param *args: args passed on to env.render(mode="ipython", ...) when show_official_viewer is True
    :param **kwargs: kwargs passed on to env.render(mode="ipython", ...) when show_official_viewer is True
    """
    data_var = None
    html = ""

    if open_koreye or show_koreye_button:
        data_var = f"koreye2022Data{get_ipython().execution_count}"
        html += f"""
<script>
function openKoreye2022(data) {{
    const tab = window.open('https://jmerle.github.io/koreye-2022/kaggle', '_blank');
    for (const ms of [100, 250, 500, 750, 1000, 1500, 2000, 3000, 4000, 5000, 10000]) {{
        setTimeout(() => tab.postMessage(data, 'https://jmerle.github.io'), ms);
    }}
}}

{data_var} = {{
    episode: {env.render(mode="json")},
    logs: {json.dumps(env.logs)},
}};
</script>
        """

    if open_koreye:
        html += f"""
<script>
if (window.location.host.endsWith('.jupyter-proxy.kaggle.net')) {{
    openKoreye2022({data_var});
}}
</script>
        """

    if show_koreye_button:
        html += f"""
<style>
.koreye-2022-button {{
    border-radius: 18px;
    cursor: pointer;
    font-family: Inter;
    font-style: normal;
    font-weight: 500;
    font-size: 14px;
    line-height: 20px;
    height: 28px;
    padding: 0 16px;
    transition: all 0.3s ease;
    width: fit-content;
    box-sizing: content-box;
}}
</style>

<button onclick="openKoreye2022({data_var})" class="koreye-2022-button">Open in Koreye 2022</button>
        """

    if html != "":
        display(HTML(html))

    if show_official_viewer:
        env.render(mode="ipython", *args, **kwargs)

In [ ]:
render_env(env, width=800, height=700)

In [ ]:
!rm -rf __pycache__/
!rm -rf *.tar.gz
!tar -czf submission.tar.gz *